### Visualise threedimodel

In this notebook we'll make an interactive map of a threedimodel using [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html).
To run this notebook locally make sure you have it installed.


As always, we first create a api-client with our authentication details.

In [ ]:
import ast
import requests
import json
from getpass import getpass
from openapi_client import ThreedimodelsApi, SimulationsApi
from threedi_api_client.threedi_api_client import ThreediApiClient
from ipyleaflet import Map, Marker, GeoJSON

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live/v3.0"
USERNAME = input("Username")
PASSWORD = getpass("Password")

config = {
    "API_HOST": API_HOST,
    "API_USERNAME": USERNAME,
    "API_PASSWORD": PASSWORD
}

api_client = ThreediApiClient(config=config)

And the threedi-model we are interested in

In [ ]:
models_api = ThreedimodelsApi(api_client)
threedi_model = models_api.threedimodels_list(name__icontains='v2_bergermeer').results[0]

print(threedi_model)

Then we display all the cells of the model on a leaflet map:

In [ ]:
extent = ast.literal_eval(threedi_model.extent_two_d)
center = (extent['coordinates'][0][1] + extent['coordinates'][1][1]) / 2.0, (extent['coordinates'][0][0] + extent['coordinates'][1][0]) / 2.0
m = Map(center=center, zoom=13)

geojson_cells_download = models_api.threedimodels_geojson_cells_download(threedi_model.id)
r = requests.get(geojson_cells_download.get_url)
geojson_data = json.loads(r.text)

geo_json = GeoJSON(
    data=geojson_data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    }
)

m.add_layer(geo_json)

display(m)